In [694]:
import csv
import tqdm
from dataclasses import dataclass
from typing import Tuple, Callable
import numpy as np
from enum import Enum
import random

In [695]:
SNACKS_FILE = "assets/data/snacks.csv"

In [696]:
MIN_VALUE = 12.0
MAX_WEIGHT = 10.0
MIN_ITEM = 2
MAX_ITEM = 4

In [697]:
@dataclass
class Item:
    name: str
    value: float
    weight: float
    
    def __str__(self):
        return f"{self.name} - {self.value} - {self.weight}"
    
    def __repr__(self):
        return str(self)

In [698]:
class types(Enum):
    single_point = 1
    two_point = 2
    uniform = 3

In [699]:
class Knapsack:
    class Chromosome:
        class Gene:
            def __init__(self, item: Item, percentage: float):
                self._item = item
                self._percentage = percentage

            @property
            def item(self):
                return self._item

            @property
            def percentage(self):
                return self._percentage

            @percentage.setter
            def percentage(self, value: float):
                self._percentage = value

            def __str__(self):
                return f"{self._item.name} - {self._percentage}"

            def __repr__(self):
                return str(self)

            @property
            def value(self):
                return self._item.value * self._percentage

            @property
            def weight(self):
                return self._item.weight * self._percentage

        def __init__(self, genes: list[Gene]):
            self._genes = genes

        @property
        def genes(self):
            return self._genes

        def __str__(self):
            return (
                "Chromosome: \n"
                + "\n".join([str(gene) for gene in self._genes])
                + f"\n------------\nValue: {self.value}\nWeight: {self.weight}\nItems: {self.num_of_items()}\n------------\n"
            )

        def __repr__(self):
            return str(self)

        @property
        def value(self):
            return sum([gene.value for gene in self._genes])

        @property
        def weight(self):
            return sum([gene.weight for gene in self._genes])

        def num_of_items(self):
            return len([gene for gene in self._genes if gene.percentage > 0])

        def __eq__(self, other):
            return self._genes == other._genes

        def __hash__(self):
            return hash(tuple(self._genes))

        def __len__(self):
            return len(self._genes)

        def __getitem__(self, index):
            return self._genes[index]

        def __setitem__(self, index, value):
            self._genes[index] = value

        def __iter__(self):
            return iter(self._genes)

    def __init__(
        self,
        items: list[Item],
        maxWeight: float,
        minItem: int,
        maxItem: int,
        fitness_function: Callable,
        selection_function: Callable,
        crossover_function: Callable,
        mutation_function: Callable,
        choose_parents_function: Callable,
        choose_survivors_function: Callable,
        population_size: int = 1000,
        mutation_rate: float = 0.1,
        crossover_rate: float = 0.7,
        max_generations: int = 1000,
        stagnation_limit: int = 100,
        type: types = types.single_point
    ):
        self._items = items
        self._maxWeight = maxWeight
        self._minItem = minItem
        self._maxItem = maxItem
        self._fitness_function = fitness_function
        self._selection_function = selection_function
        self._crossover_function = crossover_function
        self._mutation_function = mutation_function
        self._choose_parents_function = choose_parents_function
        self._choose_survivors_function = choose_survivors_function
        self._population_size = population_size
        self._mutation_rate = mutation_rate
        self._crossover_rate = crossover_rate
        self._max_generations = max_generations
        self._stagnation_limit = stagnation_limit
        self._best_fitness = -1
        self._stagnation_counter = 0
        self._population = self._generate_population()
        self._type = type

    def _generate_population(self) -> list[Chromosome]:
        return [
            self.Chromosome(
                [
                    self.Chromosome.Gene(
                        item,
                        0 if np.random.uniform(0, 1) < 0.3 else np.random.uniform(0, 1),
                    )
                    for item in self._items
                ]
            )
            for _ in range(self._population_size)
        ]

    def _fitness(self, chromosome: Chromosome) -> float:
        return self._fitness_function(chromosome)

    def _selection(self, population: list[Chromosome]) -> list[Chromosome]:
        return self._selection_function(population)

    def _crossover(
        self, parents: Tuple[Chromosome, Chromosome]
    ) -> Tuple[Chromosome, Chromosome]:
        return self._crossover_function(parents, self._crossover_rate, self._type)

    def _mutation(self, chromosome: Chromosome) -> Chromosome:
        if np.random.uniform(0, 1) < self._mutation_rate:
            return self._mutation_function(chromosome)
        return chromosome

    def _choose_parents(
        self, population: list[Chromosome]
    ) -> Tuple[Chromosome, Chromosome]:
        return self._choose_parents_function(population)

    def _choose_survivors(
        self, population: list[Chromosome], children: list[Chromosome]
    ) -> list[Chromosome]:
        return self._choose_survivors_function(population, children)

    def best_chromosome(self) -> Chromosome:
        return max(self._population, key=self._fitness)

    def _evolve(self):
        new_population = []
        for _ in range(self._population_size // 2):
            parents = self._choose_parents(self._population)
            children = self._crossover(parents)
            for child in children:
                child = self._mutation(child)
                new_population.append(child)
        self._choose_survivors(self._population, new_population)
        np.random.shuffle(self._population)

        best_chromosome_fitness = self._fitness(self.best_chromosome())
        print(f"Best fitness: {best_chromosome_fitness} - Stagnation: {self._stagnation_counter} - Best: {self._best_fitness}")
        if best_chromosome_fitness > self._best_fitness:
            self._best_fitness = best_chromosome_fitness
            self._stagnation_counter = 0
        else:
            self._stagnation_counter += 1

    def run(self, **kwargs) -> Chromosome:
        for generation in tqdm.tqdm(range(self._max_generations)):
            self._evolve()

            if kwargs.get("debug", True):
                self.print_first_3(generation)

            if (
                self._fitness(self.best_chromosome()) == 1
                or self._stagnation_counter >= self._stagnation_limit
            ):
                if self._stagnation_counter >= self._stagnation_limit:
                    print("Stagnation limit reached")
                break
        return self.best_chromosome()
    
    def print_first_3(self, generation: int = 1):
        print(f"Generation {generation}")
        for i in range(3):
            print(self._population[i])
        print("------------")

In [700]:
def fitness_function(chromosome: Knapsack.Chromosome) -> float:
    WEIGHT_COEFFICIENT = .9
    VALUE_COEFFICIENT = .7
    ITEMS_COEFFICIENT = .5
    
    weight = chromosome.weight
    value = chromosome.value
    items = chromosome.num_of_items()
    
    weight_penalty = max(0, weight - MAX_WEIGHT) / MAX_WEIGHT
    value_penalty = max(0, MIN_VALUE - value) / MIN_VALUE
    items_penalty = max(0, items - MAX_ITEM) / MAX_ITEM
    items_penalty += max(0, MIN_ITEM - items) / MIN_ITEM
    
    return 1 / (
        WEIGHT_COEFFICIENT * weight_penalty
        + VALUE_COEFFICIENT * value_penalty
        + ITEMS_COEFFICIENT * items_penalty
        + 1
    )

In [701]:
def choose_survivors(
    population: list[Knapsack.Chromosome],
    children: list[Knapsack.Chromosome],
    selection_rate: float = 0.5,
) -> list[Knapsack.Chromosome]:
    return sorted(population, key=lambda chromosome: -fitness_function(chromosome))[
        : int(len(population) * selection_rate)
    ] + sorted(children, key=lambda chromosome: -fitness_function(chromosome))[
        : int(len(population) * (1 - selection_rate))
    ]

In [702]:
def crossover_function(
    parents: Tuple[Knapsack.Chromosome, Knapsack.Chromosome],
    crossover_rate: float = 0.7,
    type: Enum = types.single_point,
) -> Tuple[Knapsack.Chromosome, Knapsack.Chromosome]:
    if np.random.uniform(0, 1) < crossover_rate:
        if type == types.single_point:
            genes1, genes2 = parents[0].genes, parents[1].genes
            crossover_point = np.random.randint(0, len(genes1))
            new_genes1 = genes1[:crossover_point] + genes2[crossover_point:]
            new_genes2 = genes2[:crossover_point] + genes1[crossover_point:]
            return (
                Knapsack.Chromosome(new_genes1),
                Knapsack.Chromosome(new_genes2),
            )
        elif type == types.two_point:
            genes1, genes2 = parents[0].genes, parents[1].genes
            crossover_point1 = np.random.randint(0, len(genes1))
            crossover_point2 = np.random.randint(crossover_point1, len(genes1))
            new_genes1 = genes1[:crossover_point1] + genes2[crossover_point1:crossover_point2] + genes1[crossover_point2:]
            new_genes2 = genes2[:crossover_point1] + genes1[crossover_point1:crossover_point2] + genes2[crossover_point2:]
            return (
                Knapsack.Chromosome(new_genes1),
                Knapsack.Chromosome(new_genes2),
            )
        elif type == types.uniform:
            genes1, genes2 = parents[0].genes, parents[1].genes
            new_genes1 = [gene1 if np.random.uniform(0, 1) < 0.5 else gene2 for gene1, gene2 in zip(genes1, genes2)]
            new_genes2 = [gene1 if np.random.uniform(0, 1) < 0.5 else gene2 for gene1, gene2 in zip(genes1, genes2)]
            return (
                Knapsack.Chromosome(new_genes1),
                Knapsack.Chromosome(new_genes2),
            )
        else:
            print("Invalid type")
            return None
    else:
        return parents

In [703]:
def choose_parents(population: list[Knapsack.Chromosome]) -> Tuple[Knapsack.Chromosome, Knapsack.Chromosome]:
    return tuple(random.sample(population, k=2))

In [704]:
def mutation_function(chromosome: Knapsack.Chromosome, mutation_rate: float = 0.1) -> Knapsack.Chromosome:
    new_genes = []
    for gene in chromosome:
        if np.random.uniform(0, 1) < mutation_rate:
            gene.percentage = np.random.uniform(0, 1)
        new_genes.append(gene)
    return Knapsack.Chromosome(new_genes)

In [705]:
def read_snacks(file: str) -> list[Item]:
    with open(file, "r") as f:
        reader = csv.reader(f)
        next(reader)
        return [Item(name, float(value), float(weight)) for name, value, weight in reader]

In [706]:
knapsack = Knapsack(
    read_snacks(SNACKS_FILE),
    MAX_WEIGHT,
    MIN_ITEM,
    MAX_ITEM,
    fitness_function,
    choose_survivors,
    crossover_function,
    mutation_function,
    choose_parents,
    choose_survivors,
    population_size=1000,
    mutation_rate=0.1,
    crossover_rate=0.7,
    max_generations=400,
    stagnation_limit=100,
    type=types.uniform
)


In [707]:
best_chromosome = knapsack.run(debug=False)

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 2/400 [00:00<00:50,  7.88it/s]

Best fitness: 0.4075812342132593 - Stagnation: 0 - Best: -1
Best fitness: 0.4075812342132593 - Stagnation: 0 - Best: 0.4075812342132593


  1%|          | 4/400 [00:00<00:41,  9.51it/s]

Best fitness: 0.4075812342132593 - Stagnation: 1 - Best: 0.4075812342132593
Best fitness: 0.4075812342132593 - Stagnation: 2 - Best: 0.4075812342132593
Best fitness: 0.4075812342132593 - Stagnation: 3 - Best: 0.4075812342132593


...

Best fitness: 0.21980193350025135 - Stagnation: 92 - Best: 0.4075812342132593
Best fitness: 0.21980193350025135 - Stagnation: 93 - Best: 0.4075812342132593


 24%|██▍       | 98/400 [00:11<00:31,  9.53it/s]

Best fitness: 0.21980193350025135 - Stagnation: 94 - Best: 0.4075812342132593
Best fitness: 0.21980193350025135 - Stagnation: 95 - Best: 0.4075812342132593
Best fitness: 0.21980193350025135 - Stagnation: 96 - Best: 0.4075812342132593


 25%|██▌       | 100/400 [00:11<00:34,  8.61it/s]

Best fitness: 0.21980193350025135 - Stagnation: 97 - Best: 0.4075812342132593
Best fitness: 0.21980193350025135 - Stagnation: 98 - Best: 0.4075812342132593
Best fitness: 0.21980193350025135 - Stagnation: 99 - Best: 0.4075812342132593
Stagnation limit reached


In [708]:
with open("assets/data/result.txt", "w") as f:
    f.write(str(best_chromosome))